In [1]:
from symred.symplectic_form import PauliwordOp, StabilizerOp, AnsatzOp, ObservableOp
from symred.S3_projection import (S3_projection, 
                                  gf2_gaus_elim, 
                                  gf2_basis_for_gf2_rref, 
                                  QubitTapering, 
                                  CS_VQE, 
                                  CheatS_VQE)
from symred.build_model import build_molecule_for_projection
from symred.utils import exact_gs_energy, quasi_model
import json
import numpy as np
from functools import reduce
import matplotlib.pyplot as plt
import openfermion as of
import openfermionpyscf as ofpyscf
from openfermion.circuits import ( uccsd_singlet_get_packed_amplitudes,
                                   uccsd_singlet_generator, uccsd_generator,
                                   uccsd_convert_amplitude_format)
from cirq.testing import random_unitary
from itertools import combinations, product

import warnings
warnings.filterwarnings("ignore")

In [2]:
with open('data/molecule_data.json', 'r') as jfile:
    molecule_geometries = json.load(jfile)
print(molecule_geometries.keys())

dict_keys(['H2_3-21G_SINGLET', 'H6_STO-3G_SINGLET', 'H2_6-31G_SINGLET', 'H2_6-311G_SINGLET', 'H3+_STO-3G_SINGLET', 'H3+_3-21G_SINGLET', 'HeH+_3-21G_SINGLET', 'HeH+_6-311G_SINGLET', 'H2O_STO-3G_SINGLET', 'BeH+_STO-3G_SINGLET', 'LiH_STO-3G_SINGLET', 'CH+_STO-3G_SINGLET', 'HF_STO-3G_SINGLET', 'B+_STO-3G_SINGLET', 'B_STO-3G_DOUBLET', 'N_STO-3G_QUARTET', 'OH-_STO-3G_SINGLET', 'O_STO-3G_TRIPLET', 'CH2_STO-3G_TRIPLET', 'BeH2_STO-3G_SINGLET', 'Be_STO-3G_SINGLET', 'C_STO-3G_TRIPLET', 'NH_STO-3G_SINGLET', 'Ne_STO-3G_SINGLET', 'F_STO-3G_DOUBLET', 'Li_STO-3G_DOUBLET', 'BH_STO-3G_SINGLET', 'NeH+_STO-3G_SINGLET', 'NH2+_STO-3G_SINGLET', 'BH2+_STO-3G_SINGLET', 'HCl_STO-3G_SINGLET', 'H4_STO-3G_SINGLET', 'NH3_STO-3G_SINGLET', 'F2_STO-3G_SINGLET', 'HCN_STO-3G_SINGLET', 'CH4_STO-3G_SINGLET', 'CH3NHCH3_STO-3G_SINGLET', 'CH3CH2NH2_STO-3G_SINGLET', 'CH3CH2OH_STO-3G_SINGLET', 'CH3OH_STO-3G_SINGLET', 'C2H6_STO-3G_SINGLET', 'CH3CN_STO-3G_SINGLET', 'CH3CHO_STO-3G_SINGLET', 'CH3CHOHCH3_STO-3G_SINGLET', 'CHONH2_ST

In [10]:
H2O_ham_dict = {'IIIIIIIIII': (-46.18174224503647+0j), 'IIIIIIIIIZ': (0.8051783030466303+0j), 'IIIIIIIIZI': (0.80517830304663+0j), 'IIIIIIIIZZ': (0.31372270245504164+0j), 'IIIIIIIZII': (0.7771051535723017+0j), 'IIIIIIIZIZ': (0.11275000398784477+0j), 'IIIIIIIZZI': (0.14165695082321583+0j), 'IIIIIIZIII': (0.7771051535723017+0j), 'IIIIIIZIIZ': (0.14165695082321583+0j), 'IIIIIIZIZI': (0.11275000398784477+0j), 'IIIIIIZZII': (0.14926548536877954+0j), 'IIIIIZIIII': (2.743259461270587+0j), 'IIIIIZIIIZ': (0.3068383859034868+0j), 'IIIIIZIIZI': (0.3068383859034868+0j), 'IIIIIZIZII': (0.2847518837060843+0j), 'IIIIIZZIII': (0.2847518837060843+0j), 'IIIIZIIIII': (1.318485324793361+0j), 'IIIIZIIIIZ': (0.13521837630461306+0j), 'IIIIZIIIZI': (0.15239274116835275+0j), 'IIIIZIIZII': (0.12035268674229192+0j), 'IIIIZIZIII': (0.13721425345353316+0j), 'IIIIZZIIII': (0.3511935221416017+0j), 'IIIZIIIIII': (1.318485324793361+0j), 'IIIZIIIIIZ': (0.15239274116835275+0j), 'IIIZIIIIZI': (0.13521837630461306+0j), 'IIIZIIIZII': (0.13721425345353316+0j), 'IIIZIIZIII': (0.12035268674229192+0j), 'IIIZIZIIII': (0.3511935221416017+0j), 'IIIZZIIIII': (0.19652924197864333+0j), 'IIZIIIIIII': (1.2135089673725632+0j), 'IIZIIIIIIZ': (0.22984584849588102+0j), 'IIZIIIIIZI': (0.30589431775894094+0j), 'IIZIIIIIZZ': (1.2135089673725632-0j), 'IIZIIIIZII': (0.12521733640631744+0j), 'IIZIIIIZZZ': (0.14298602258298954-0j), 'IIZIIIZIII': (0.14298602258298954+0j), 'IIZIIIZIZZ': (0.12521733640631744-0j), 'IIZIIZIIII': (0.3076563919366747+0j), 'IIZIIZIIZZ': (0.3076563919366747+0j), 'IIZIZIIIII': (0.1382045498790267+0j), 'IIZIZIIIZZ': (0.14993419791017615-0j), 'IIZIZZIZIZ': (-0.23655337829053713+0j), 'IIZZIIIIII': (0.14993419791017615+0j), 'IIZZIIIIZZ': (0.1382045498790267-0j), 'IIZZIZZIIZ': (-0.23655337829053713+0j), 'IZIIIIIIII': (1.6577822216057931+0j), 'IZIIIIIIIZ': (0.1408093440610478+0j), 'IZIIIIIIZI': (0.15623959624985947+0j), 'IZIIIIIZII': (0.12808267899511272+0j), 'IZIIIIZIII': (0.1535927430066858+0j), 'IZIIIZIIII': (0.33757611993210745+0j), 'IZIIZIIIII': (0.13815580367622055+0j), 'IZIIZZIZIZ': (-0.19752915409654886+0j), 'IZIIZZIZZI': (-0.20027869865695302+0j), 'IZIZIIIIII': (0.16911013247662537+0j), 'IZZIIIIIII': (0.12528022881511333+0j), 'IZZIIIIIZZ': (0.16151687457022879-0j), 'IZZIIZIZIZ': (-0.24368338824184604+0j), 'IZZIZIIZIZ': (-0.5511556266732167+0j), 'IZZIZZIIIZ': (-0.19272564381806373+0j), 'IZZIZZIZII': (-0.2123371526292752+0j), 'IZZIZZIZIZ': (-12.413911714677475+0j), 'IZZIZZIZZZ': (-0.2177188659495859+0j), 'IZZIZZZZIZ': (-0.20059914588883654+0j), 'IZZZZZIZIZ': (-0.2506518981252438+0j), 'ZIIIIIIIII': (1.657782221605793+0j), 'ZIIIIIIIIZ': (0.15623959624985947+0j), 'ZIIIIIIIZI': (0.1408093440610478+0j), 'ZIIIIIIZII': (0.1535927430066858+0j), 'ZIIIIIZIII': (0.12808267899511272+0j), 'ZIIIIZIIII': (0.33757611993210745+0j), 'ZIIIZIIIII': (0.16911013247662537+0j), 'ZIIZIIIIII': (0.13815580367622055+0j), 'ZIIZIZZIIZ': (-0.20027869865695302+0j), 'ZIIZIZZIZI': (-0.19752915409654886+0j), 'ZIZIIIIIII': (0.16151687457022879+0j), 'ZIZIIIIIZZ': (0.12528022881511333-0j), 'ZIZIIZZIIZ': (-0.24368338824184604+0j), 'ZIZZIIZIIZ': (-0.5511556266732167+0j), 'ZIZZIZIIIZ': (-0.19272564381806373+0j), 'ZIZZIZZIII': (-0.2177188659495859+0j), 'ZIZZIZZIIZ': (-12.413911714677475+0j), 'ZIZZIZZIZZ': (-0.2123371526292752+0j), 'ZIZZIZZZIZ': (-0.20059914588883654+0j), 'ZIZZZZZIIZ': (-0.2506518981252438+0j), 'ZZIIIIIIII': (0.18211285620237594+0j), 'ZZIZZIZZII': (1.1861011973660436+0j), 'ZZZIZZIZIZ': (-0.25108006382755094+0j), 'ZZZZIZZIIZ': (-0.25108006382755094+0j), 'IIIZZIZZIX': (-0.003841431594772443+0j), 'IZZZIZZIIX': (-0.003841431594772443+0j), 'ZIIIZZIZZX': (-0.003841431594772443+0j), 'ZZZIIIIIZX': (-0.003841431594772443+0j), 'IIIIZIZZXZ': (-0.039505364753839686+0j), 'IIIZIIZZXZ': (-0.0402324683035052+0j), 'IIIZZIIZXZ': (-0.025624096155624877+0j), 'IIIZZIZIXZ': (-0.00942913948371769+0j), 'IIIZZIZZXI': (-0.0007523422027558499+0j), 'IIIZZIZZXZ': (-0.36898679109307975+0j), 'IIIZZZZZXZ': (-0.08882030288492249+0j), 'IIZIZIZZXI': (0.039505364753839686+0j), 'IIZZIIZZXI': (0.0402324683035052+0j), 'IIZZZIIZXI': (0.025624096155624877+0j), 'IIZZZIZIXI': (0.00942913948371769+0j), 'IIZZZIZZXI': (0.36898679109307975+0j), 'IIZZZIZZXZ': (0.0007523422027558499+0j), 'IIZZZZZZXI': (0.08882030288492249+0j), 'IZIZIZZIXZ': (-0.09059252882998142+0j), 'IZIZZIZZXZ': (-0.03443625643509166+0j), 'IZZZIZZIXI': (0.09059252882998142-0j), 'IZZZZIZZXI': (0.03443625643509166+0j), 'ZIIIZZIZXZ': (-0.08675109723520896+0j), 'ZIIZZIZZXZ': (-0.02445414497679984+0j), 'ZIZIZZIZXI': (0.08675109723520896-0j), 'ZIZZZIZZXI': (0.02445414497679984+0j), 'IIIIIIIIYY': (-0.005381713320310727+0j), 'IZZIZZIZYY': (-0.005381713320310727+0j), 'ZIZZIZZIXX': (0.005381713320310727+0j), 'ZZIZZIZZXX': (0.005381713320310727+0j), 'IIIIIIIXZZ': (-0.05994823321175161+0j), 'IIZZIZZXZI': (0.0002036534971912864+0j), 'IZIZZIZXZZ': (0.0002036534971912864+0j), 'IZZIZZIXZI': (-0.05994823321175161+0j), 'ZIIZIZZXIZ': (-3.803093727859245e-05+0j), 'ZIIZIZZXZI': (0.000748383645029448+0j), 'ZIIZZIZXZZ': (-0.001469495273070372+0j), 'ZIZIIZZXZI': (0.0048261344504113915+0j), 'ZIZZIIZXZI': (0.007091603943013649+0j), 'ZIZZIZIXZI': (-0.00207926825387612+0j), 'ZIZZIZZXII': (0.0013053731706706203+0j), 'ZIZZIZZXZI': (0.07314533074509928+0j), 'ZIZZIZZXZZ': (0.004387061336512373+0j), 'ZIZZZZZXZI': (0.005447997131394088+0j), 'ZZIIZIZXZZ': (0.0048261344504113915+0j), 'ZZIZIIZXZZ': (0.005447997131394088+0j), 'ZZIZZIIXZZ': (-0.00207926825387612+0j), 'ZZIZZIZXIZ': (0.0013053731706706203+0j), 'ZZIZZIZXZI': (0.004387061336512373+0j), 'ZZIZZIZXZZ': (0.07314533074509928+0j), 'ZZIZZZZXZZ': (0.007091603943013649+0j), 'ZZZZIZZXZI': (-0.001469495273070372+0j), 'ZZZZZIZXII': (-3.803093727859245e-05+0j), 'ZZZZZIZXZZ': (0.000748383645029448+0j), 'IIZZZIZXIX': (-0.00232274945451925+0j), 'IIZZZIZYIY': (-0.00232274945451925+0j), 'ZIZIZZIXZX': (-0.00232274945451925+0j), 'ZIZIZZIYZY': (-0.00232274945451925+0j), 'ZIIIZZIYYI': (0.0017551322035335862+0j), 'ZIZIZZIYYZ': (-0.0017551322035335862+0j), 'ZZIIIIIYYI': (-0.0017551322035335862+0j), 'ZZZIIIIYYZ': (0.0017551322035335862+0j), 'IIIIIIIXYY': (-0.0030816881658417518+0j), 'IIIIIIIYYX': (0.0030816881658417518+0j), 'ZIZZIZZXXX': (0.0030816881658417518+0j), 'ZIZZIZZYXY': (0.0030816881658417518+0j), 'IIIIIIXIZZ': (0.00207926825387612+0j), 'IIIIIIXZIZ': (-0.004387061336512373+0j), 'IIIIIIXZZI': (-0.0013053731706706203+0j), 'IIIIIIXZZZ': (-0.07314533074509928+0j), 'IIIIIZXZZZ': (-0.007091603943013649+0j), 'IIIIZIXZZZ': (-0.0048261344504113915+0j), 'IIIZIIXZZZ': (-0.005447997131394088+0j), 'IIZIIIXZII': (-0.000748383645029448+0j), 'IIZIIIXZZZ': (3.803093727859245e-05+0j), 'IIZZIZXZZI': (0.001469495273070372+0j), 'IZIIIIXZZZ': (-0.0002036534971912864+0j), 'IZZIZZXIZI': (0.05994823321175161+0j), 'ZIIIIIXZZZ': (0.001469495273070372+0j), 'ZIIZIZXZIZ': (-0.000748383645029448+0j), 'ZIIZIZXZZI': (3.803093727859245e-05+0j), 'ZIZIIZXZZI': (-0.005447997131394088+0j), 'ZIZZIIXZZI': (-0.007091603943013649+0j), 'ZIZZIZXIZI': (0.00207926825387612+0j), 'ZIZZIZXZII': (-0.004387061336512373+0j), 'ZIZZIZXZZI': (-0.07314533074509928+0j), 'ZIZZIZXZZZ': (-0.0013053731706706203+0j), 'ZIZZZZXZZI': (-0.0048261344504113915+0j), 'ZZIZZIXIZZ': (0.05994823321175161+0j), 'ZZZZIZXZZI': (-0.0002036534971912864+0j), 'ZIIIZZYIIY': (0.0009684913838387863+0j), 'ZIZIZZXIZX': (0.0009684913838387863-0j), 'ZZIIIIXZZX': (0.0009684913838387863-0j), 'ZZZIIIYZIY': (0.0009684913838387863+0j), 'IIIZZIYIYI': (0.0027236235873723726-0j), 'IIZZZIXIXZ': (-0.001354258070680464+0j), 'IIZZZIYIYZ': (-0.0040778816580528355+0j), 'ZIIIZZYIYI': (-0.0027236235873723726+0j), 'ZIZIZZXIXZ': (0.001354258070680464-0j), 'ZIZIZZYIYZ': (0.0040778816580528355-0j), 'ZIZZIZXZYY': (0.0030816881658417518+0j), 'ZIZZIZYZXY': (-0.0030816881658417518+0j), 'ZZIZZIXIYY': (0.0030816881658417518+0j), 'ZZIZZIYIXY': (-0.0030816881658417518+0j), 'IIIIIIYYII': (-0.007873502070772793+0j), 'IZZIZZYYIZ': (-0.007873502070772793+0j), 'ZIZZIZYYIZ': (-0.007873502070772793+0j), 'ZZIZZIYYII': (-0.007873502070772793+0j), 'IIIZZIYXIY': (0.01619495667190719-0j), 'IIIZZIYYIX': (-0.01619495667190719+0j), 'IIZZZIXXZX': (0.01619495667190719+0j), 'IIZZZIXYZY': (0.01619495667190719+0j), 'IIIIIIXXYY': (-0.028906946835371088+0j), 'IIIIIIXYYX': (0.028906946835371088+0j), 'IIIIIIYXXY': (0.028906946835371088+0j), 'IIIIIIYYXX': (-0.028906946835371088+0j), 'IIIIIXIIII': (-0.006512028610684347+0j), 'IZZIZXIZIZ': (-0.006512028610684347+0j), 'ZIZZIXZIIZ': (-0.006512028610684347+0j), 'ZZIZZXZZII': (-0.006512028610684347+0j), 'IIIZZXZZIX': (0.005921448772451484+0j), 'IIIZZYZZIY': (0.005921448772451484+0j), 'IIZZZXZZZX': (0.005921448772451484+0j), 'IIZZZYZZZY': (0.005921448772451484+0j), 'IIIIIXIIXX': (-0.006088804229429438+0j), 'IIIIIXIIYY': (0.006088804229429438-0j), 'IIIIIYIIXY': (-0.006088804229429438+0j), 'IIIIIYIIYX': (-0.006088804229429438+0j), 'IIIIIXIXZZ': (-0.003967451072256146+0j), 'IIIIIYIYZZ': (-0.003967451072256146+0j), 'ZIZZIXZXZI': (-0.003967451072256146+0j), 'ZIZZIYZYZI': (-0.003967451072256146+0j), 'ZIZZIXXZZI': (0.003967451072256146-0j), 'ZIZZIYYZZI': (0.003967451072256146+0j), 'ZZIZZXXIZZ': (0.003967451072256146-0j), 'ZZIZZYYIZZ': (0.003967451072256146+0j), 'IIIIIXXXII': (-0.00969149937317402+0j), 'IIIIIXYYII': (0.00969149937317402-0j), 'IIIIIYXYII': (-0.00969149937317402+0j), 'IIIIIYYXII': (-0.00969149937317402+0j), 'IIIIXIZZZZ': (0.045871829829892984-0j), 'IIZZXZIZZI': (-0.0040356005021943515+0j), 'IZIZXIIIZZ': (-0.0040356005021943515+0j), 'IZZIXZZIZI': (0.045871829829892984-0j), 'ZIIZXIIIZZ': (-0.0017294790014541555+0j), 'ZIIZXZIZIZ': (-0.001619837015612757+0j), 'ZIIZXZIZZI': (-0.0024816349993497504+0j), 'ZIZIXZIZZI': (0.002871658321452742-0j), 'ZIZZXIIZZI': (-0.005918980791196522+0j), 'ZIZZXZIIZI': (-0.005357732323146429+0j), 'ZIZZXZIZII': (-0.0010404826273716214+0j), 'ZIZZXZIZZI': (-0.04270951779188808+0j), 'ZIZZXZIZZZ': (-0.004354912124887974+0j), 'ZIZZXZZZZI': (-0.005323072473017693+0j), 'ZZIIXIIIZZ': (0.002871658321452742-0j), 'ZZIZXIIIIZ': (-0.0010404826273716214+0j), 'ZZIZXIIIZI': (-0.004354912124887974+0j), 'ZZIZXIIIZZ': (-0.04270951779188808+0j), 'ZZIZXIIZZZ': (-0.005357732323146429+0j), 'ZZIZXIZIZZ': (-0.005323072473017693+0j), 'ZZIZXZIIZZ': (-0.005918980791196522+0j), 'ZZZZXIIIII': (-0.001619837015612757+0j), 'ZZZZXIIIZZ': (-0.0024816349993497504+0j), 'ZZZZXZIZZI': (-0.0017294790014541555+0j), 'IIZZXIIIIX': (0.0024136227932753133-0j), 'IIZZYIIIIY': (0.0024136227932753133-0j), 'ZIZIXZZIZX': (0.0024136227932753133+0j), 'ZIZIYZZIZY': (0.0024136227932753133+0j), 'ZIIIYZZIYI': (0.00021175546627918873-0j), 'ZIZIYZZIYZ': (-0.00021175546627918873+0j), 'ZZIIYIZZYI': (-0.00021175546627918873+0j), 'ZZZIYIZZYZ': (0.00021175546627918873-0j), 'IIIIXIZZYY': (0.003314429497516353-0j), 'IIIIYIZZYX': (-0.003314429497516353+0j), 'ZIZZXZIZXX': (-0.003314429497516353+0j), 'ZIZZYZIZXY': (-0.003314429497516353+0j), 'IIIIXIIXII': (-0.011074296933117133+0j), 'IIIIXIZXII': (-0.22116656188019804+0j), 'IIIIXIZXIZ': (-0.024744999856263626+0j), 'IIIIXIZXZI': (-0.010310591690493262+0j), 'IIIIXZZXII': (-0.05791287308384983+0j), 'IIIIYIIYII': (-0.011074296933117133+0j), 'IIIIYIZYII': (-0.22116656188019804+0j), 'IIIIYIZYIZ': (-0.024744999856263626+0j), 'IIIIYIZYZI': (-0.010310591690493262+0j), 'IIIIYZZYII': (-0.05791287308384983+0j), 'IIIZXIZXII': (-0.030100938424054008+0j), 'IIIZYIZYII': (-0.030100938424054008+0j), 'IIZIXIZXII': (-0.017814808777335257+0j), 'IIZIXIZXZZ': (-0.010736002999738665+0j), 'IIZIYIZYII': (-0.017814808777335257+0j), 'IIZIYIZYZZ': (-0.010736002999738665+0j), 'IZIIXIZXII': (-0.018426482046492026+0j), 'IZIIYIZYII': (-0.018426482046492026+0j), 'IZZIXZZXIZ': (-0.05443111876673117+0j), 'IZZIYZZYIZ': (-0.05443111876673117+0j), 'ZIIIXIZXII': (-0.023614562486533458+0j), 'ZIIIYIZYII': (-0.023614562486533458+0j), 'ZIZZXZIXIZ': (0.054260020689901815-0j), 'ZIZZYZIYIZ': (0.054260020689901815-0j), 'IIIZXIIXXZ': (-0.019337553877385297+0j), 'IIIZYIIYXZ': (-0.019337553877385297+0j), 'IIZZXIIXXI': (0.019337553877385297-0j), 'IIZZYIIYXI': (0.019337553877385297-0j), 'IIIIXIXIII': (0.00017109807682935495-0j), 'IZZIXZXZIZ': (0.00017109807682935495-0j), 'ZIZZXZXIIZ': (0.00017109807682935495-0j), 'ZZIZXIXZII': (0.00017109807682935495-0j), 'IIIZXIXZIX': (0.011030861199352059+0j), 'IIIZYIXZIY': (0.011030861199352059+0j), 'IIZZXIYZZY': (0.011030861199352059-0j), 'IIZZYIYZZX': (-0.011030861199352059+0j), 'IIIIXIXIXX': (0.014434408165770362-0j), 'IIIIXIYIYX': (0.014434408165770362-0j), 'IIIIYIXIXY': (0.014434408165770362-0j), 'IIIIYIYIYY': (0.014434408165770362-0j), 'IIIIXIYYZZ': (3.465985012873635e-05-0j), 'IIIIYIYXZZ': (-3.465985012873635e-05+0j), 'ZIZZXZYYZI': (3.465985012873635e-05-0j), 'ZIZZYZYXZI': (-3.465985012873635e-05+0j), 'IIIIXXZZZZ': (0.0033627525441588726+0j), 'IIIIYYZZZZ': (0.0033627525441588726+0j), 'ZIZZXXIZZI': (0.0033627525441588726-0j), 'ZIZZYYIZZI': (0.0033627525441588726-0j), 'IIIIXXXIII': (-0.000505344691244693+0j), 'IIIIXYYIII': (-0.000505344691244693+0j), 'IIIIYXYIII': (0.000505344691244693-0j), 'IIIIYYXIII': (-0.000505344691244693+0j), 'IIIXIIZZZZ': (-0.002871658321452742+0j), 'IIIXZIIZZZ': (0.005357732323146429-0j), 'IIIXZIZIZZ': (0.005323072473017693-0j), 'IIIXZIZZIZ': (0.004354912124887974-0j), 'IIIXZIZZZI': (0.0010404826273716214-0j), 'IIIXZIZZZZ': (0.042709517791888064-0j), 'IIIXZZZZZZ': (0.005918980791196522+0j), 'IIZXZIZZII': (0.0024816349993497504+0j), 'IIZXZIZZZZ': (0.001619837015612757-0j), 'IIZXZZIZZI': (0.0017294790014541555-0j), 'IZIXZIZZZZ': (0.0040356005021943515-0j), 'IZZXIZZIZI': (-0.045871829829892984+0j), 'ZIIXZIZZZZ': (0.0017294790014541555-0j), 'ZIIXZZIZIZ': (0.0024816349993497504+0j), 'ZIIXZZIZZI': (0.001619837015612757-0j), 'ZIZXIZIZZI': (-0.002871658321452742+0j), 'ZIZXZIIZZI': (0.005918980791196522+0j), 'ZIZXZZIIZI': (0.005323072473017693-0j), 'ZIZXZZIZII': (0.004354912124887974-0j), 'ZIZXZZIZZI': (0.042709517791888064-0j), 'ZIZXZZIZZZ': (0.0010404826273716214-0j), 'ZIZXZZZZZI': (0.005357732323146429-0j), 'ZZIXIIIIZZ': (-0.045871829829892984+0j), 'ZZZXZZIZZI': (0.0040356005021943515-0j), 'ZIIYIZZIIY': (-0.0012470236322305668+0j), 'ZIZXIZZIZX': (-0.0012470236322305668+0j), 'ZZIXZIZZZX': (-0.0012470236322305668+0j), 'ZZZYZIZZIY': (-0.0012470236322305668+0j), 'IIIYIIIIYI': (-0.0010352681659513783+0j), 'IIZXIIIIXZ': (0.001166599161044747-0j), 'IIZYIIIIYZ': (0.002201867326996125-0j), 'ZIIYIZZIYI': (0.0010352681659513783-0j), 'ZIZXIZZIXZ': (-0.001166599161044747+0j), 'ZIZYIZZIYZ': (-0.002201867326996125+0j), 'ZIZXZZIZYY': (-0.0033144294975163534+0j), 'ZIZYZZIZXY': (0.0033144294975163534-0j), 'ZZIXIIIIYY': (-0.0033144294975163534+0j), 'ZZIYIIIIXY': (0.0033144294975163534-0j), 'IIIYZIZYII': (0.00017109807682935495-0j), 'IZZYIZZYIZ': (0.00017109807682935495-0j), 'ZIZYZZIYIZ': (0.00017109807682935495-0j), 'ZZIYIIIYII': (0.00017109807682935495-0j), 'IIIYIIIXIY': (-0.011767712432203212+0j), 'IIIYIIIYIX': (0.011767712432203212+0j), 'IIZXIIIXZX': (-0.011767712432203212+0j), 'IIZXIIIYZY': (-0.011767712432203212+0j), 'IIIXZIZXYY': (0.014434408165770362-0j), 'IIIXZIZYYX': (-0.014434408165770362+0j), 'IIIYZIZXXY': (-0.014434408165770362+0j), 'IIIYZIZYXX': (0.014434408165770362-0j), 'IIIXIIXIII': (-0.030100938424054008+0j), 'IIIXZIXIII': (-0.221166561880198+0j), 'IIIXZIXIIZ': (-0.010310591690493262+0j), 'IIIXZIXIZI': (-0.024744999856263626+0j), 'IIIXZIXZII': (-0.011074296933117133+0j), 'IIIXZZXIII': (-0.05791287308384983+0j), 'IIIYIIYIII': (-0.030100938424054008+0j), 'IIIYZIYIII': (-0.221166561880198+0j), 'IIIYZIYIIZ': (-0.010310591690493262+0j), 'IIIYZIYIZI': (-0.024744999856263626+0j), 'IIIYZIYZII': (-0.011074296933117133+0j), 'IIIYZZYIII': (-0.05791287308384983+0j), 'IIZXZIXIII': (-0.010736002999738665+0j), 'IIZXZIXIZZ': (-0.017814808777335257+0j), 'IIZYZIYIII': (-0.010736002999738665+0j), 'IIZYZIYIZZ': (-0.017814808777335257+0j), 'IZIXZIXIII': (-0.023614562486533458+0j), 'IZIYZIYIII': (-0.023614562486533458+0j), 'IZZXIZXZIZ': (0.054260020689901815-0j), 'IZZYIZYZIZ': (0.054260020689901815-0j), 'ZIIXZIXIII': (-0.018426482046492026+0j), 'ZIIYZIYIII': (-0.018426482046492026+0j), 'ZIZXZZXIIZ': (-0.05443111876673117+0j), 'ZIZYZZYIIZ': (-0.05443111876673117+0j), 'IIIXIIXZXZ': (-0.008306692678033238+0j), 'IIIYIIXZYZ': (-0.0007368512328511548+0j), 'IIIYIIYZXZ': (-0.007569841445182085+0j), 'IIZXIIXZXI': (0.007569841445182085-0j), 'IIZXIIYZYI': (-0.0007368512328511548+0j), 'IIZYIIYZXI': (0.008306692678033238-0j), 'ZIZXZZXXZI': (3.465985012873635e-05-0j), 'ZIZYZZYXZI': (3.465985012873635e-05-0j), 'ZZIXIIXXZZ': (3.465985012873635e-05-0j), 'ZZIYIIYXZZ': (3.465985012873635e-05-0j), 'ZIZXZXIZZI': (-0.0033627525441588726+0j), 'ZIZYZYIZZI': (-0.0033627525441588726+0j), 'ZZIXIXIIZZ': (-0.0033627525441588726+0j), 'ZZIYIYIIZZ': (-0.0033627525441588726+0j), 'IIIXZXZXII': (0.000505344691244693-0j), 'IIIXZYZYII': (0.000505344691244693-0j), 'IIIYZXZYII': (-0.000505344691244693+0j), 'IIIYZYZXII': (0.000505344691244693+0j), 'IIIYYIIIII': (-0.006968509883397754+0j), 'IZZYYZIZIZ': (-0.006968509883397754+0j), 'ZIZYYZZIIZ': (-0.006968509883397754+0j), 'ZZIYYIZZII': (-0.006968509883397754+0j), 'IIIYXIZZIY': (-0.0007271035496655186+0j), 'IIIYYIZZIX': (0.0007271035496655186-0j), 'IIZXXIZZZX': (-0.0007271035496655186+0j), 'IIZXYIZZZY': (-0.0007271035496655186+0j), 'IIIXXIIIYY': (-0.0171743648637397+0j), 'IIIXYIIIYX': (0.0171743648637397+0j), 'IIIYXIIIXY': (0.0171743648637397+0j), 'IIIYYIIIXX': (-0.0171743648637397+0j), 'IIIXXIIXZZ': (0.0006218626809826967+0j), 'IIIXYIIYZZ': (0.0006218626809826967+0j), 'ZIZXXZZXZI': (0.0006218626809826967+0j), 'ZIZXYZZYZI': (0.0006218626809826967+0j), 'ZIZXYZYZZI': (-0.0006218626809826967+0j), 'ZIZYYZXZZI': (0.0006218626809826967+0j), 'ZZIXYIYIZZ': (-0.0006218626809826967+0j), 'ZZIYYIXIZZ': (0.0006218626809826967+0j), 'IIIXXIYYII': (-0.01686156671124125+0j), 'IIIXYIYXII': (0.01686156671124125+0j), 'IIIYXIXYII': (0.01686156671124125+0j), 'IIIYYIXXII': (-0.01686156671124125+0j), 'IIIXXXIIII': (-0.014023611932222154+0j), 'IIIXYYIIII': (-0.014023611932222154+0j), 'IIIYXYIIII': (-0.014023611932222154+0j), 'IIIYYXIIII': (0.014023611932222154-0j), 'IIXIIIIIZZ': (-0.002749544560404149+0j), 'IZXIZZIZZI': (-0.002749544560404149+0j), 'ZIXZIZZIIZ': (-0.002749544560404149+0j), 'ZZXZZIZZII': (-0.002749544560404149+0j), 'IIXIZIZZZX': (-0.0402324683035052+0j), 'IIXZIIZZZX': (-0.039505364753839686+0j), 'IIXZZIIZZX': (-0.00942913948371769+0j), 'IIXZZIZIZX': (-0.025624096155624877+0j), 'IIXZZIZZIX': (-0.0007523422027558499+0j), 'IIXZZIZZZX': (-0.36898679109307975+0j), 'IIXZZZZZZX': (-0.08882030288492249+0j), 'IIYIZIZZZY': (-0.0402324683035052+0j), 'IIYZIIZZZY': (-0.039505364753839686+0j), 'IIYZZIIZZY': (-0.00942913948371769+0j), 'IIYZZIZIZY': (-0.025624096155624877+0j), 'IIYZZIZZIY': (-0.0007523422027558499+0j), 'IIYZZIZZZY': (-0.36898679109307975+0j), 'IIYZZZZZZY': (-0.08882030288492249+0j), 'IZXZIZZIZX': (-0.08675109723520896+0j), 'IZXZZIZZZX': (-0.02445414497679984+0j), 'IZYZIZZIZY': (-0.08675109723520896+0j), 'IZYZZIZZZY': (-0.02445414497679984+0j), 'ZIXIZZIZZX': (-0.09059252882998142+0j), 'ZIXZZIZZZX': (-0.03443625643509166+0j), 'ZIYIZZIZZY': (-0.09059252882998142+0j), 'ZIYZZIZZZY': (-0.03443625643509166+0j), 'IIXZZIZZXI': (-0.003841431594772443+0j), 'IZXZIZZIXZ': (-0.003841431594772443+0j), 'ZIYIZZIZYZ': (0.003841431594772443-0j), 'ZZYIIIIIYI': (0.003841431594772443-0j), 'IIXIIIIIYY': (0.07604846926305996+0j), 'IIYIIIIIYX': (-0.07604846926305996+0j), 'IIXIIIIXZZ': (-0.0007864145823080404+0j), 'IIYIIIIYZZ': (-0.0007864145823080404+0j), 'ZIXZIZZXIZ': (-0.0007864145823080404+0j), 'ZIYZIZZYIZ': (-0.0007864145823080404+0j), 'ZIXIZZIYIY': (0.0027236235873723726-0j), 'ZIYIZZIXIY': (0.001354258070680464-0j), 'ZIYIZZIYIX': (-0.0040778816580528355+0j), 'ZZXIIIIYIY': (-0.0027236235873723726+0j), 'ZZYIIIIXIY': (-0.001354258070680464+0j), 'ZZYIIIIYIX': (0.0040778816580528355-0j), 'IIXZZIZXXZ': (-0.0009684913838387863+0j), 'IIYZZIZYXZ': (-0.0009684913838387863+0j), 'ZIXIZZIYYI': (-0.0009684913838387863+0j), 'ZIYIZZIXYI': (0.0009684913838387863-0j), 'ZIXZIZXZZI': (0.0007864145823080404-0j), 'ZIYZIZYZIZ': (0.0007864145823080404+0j), 'ZZXZZIXIZZ': (0.0007864145823080404-0j), 'ZZYZZIYIII': (0.0007864145823080404+0j), 'IIXZZIYIIY': (0.0017551322035335862-0j), 'IIYZZIYIIX': (-0.0017551322035335862+0j), 'ZIXIZZYIIY': (-0.0017551322035335862+0j), 'ZIYIZZYIIX': (0.0017551322035335862-0j), 'ZIYIZZXIYI': (-0.00232274945451925+0j), 'ZIYIZZYIXI': (0.00232274945451925-0j), 'ZZYIIIXZYZ': (-0.00232274945451925+0j), 'ZZYIIIYZXZ': (0.00232274945451925-0j), 'IIXIIIXXZZ': (-0.017768686176672096+0j), 'IIXIIIYYII': (0.017768686176672096-0j), 'IIYIIIXYZZ': (-0.017768686176672096+0j), 'IIYIIIYXII': (-0.017768686176672096+0j), 'IIXZZIXYYI': (0.016194956671907193-0j), 'IIXZZIYYXI': (-0.016194956671907193+0j), 'IIYZZIXXYI': (-0.016194956671907193+0j), 'IIYZZIYXXI': (0.016194956671907193-0j), 'IIXIIXIIII': (0.007221016602947202-0j), 'IIXIIXIIZZ': (0.007221016602947202-0j), 'IIYIIYIIII': (0.007221016602947202-0j), 'IIYIIYIIZZ': (0.007221016602947202-0j), 'IIXZZXZZXI': (0.005921448772451484+0j), 'IIXZZYZZYI': (0.005921448772451484-0j), 'IIYZZXZZYI': (-0.005921448772451484+0j), 'IIYZZYZZXI': (0.005921448772451484+0j), 'IIXIXIZZZZ': (0.0008617979837369933+0j), 'IIYIYIZZZZ': (0.0008617979837369933+0j), 'ZIXZXZIZIZ': (0.0008617979837369933-0j), 'ZIYZYZIZIZ': (0.0008617979837369933-0j), 'ZIXIYZZIIY': (-0.0010352681659513783+0j), 'ZIYIXZZIIY': (-0.001166599161044747+0j), 'ZIYIYZZIIX': (0.002201867326996125+0j), 'ZZXIYIZZIY': (0.0010352681659513783+0j), 'ZZYIXIZZIY': (0.001166599161044747+0j), 'ZZYIYIZZIX': (-0.002201867326996125+0j), 'IIXZXIIIXZ': (0.0012470236322305668+0j), 'IIYZYIIIXZ': (0.0012470236322305668+0j), 'ZIXIYZZIYI': (0.0012470236322305668+0j), 'ZIYIXZZIYI': (-0.0012470236322305668+0j), 'IIXZXIIXZX': (-0.007569841445182085+0j), 'IIXZXIIYZY': (0.0007368512328511548+0j), 'IIXZYIIYZX': (-0.008306692678033238+0j), 'IIYZXIIXZY': (-0.008306692678033238+0j), 'IIYZYIIXZX': (0.0007368512328511548+0j), 'IIYZYIIYZY': (-0.007569841445182085+0j), 'IIXIXIXIII': (-0.007078805777596593+0j), 'IIXIYIYIZZ': (0.007078805777596593-0j), 'IIYIXIYIZZ': (-0.007078805777596593+0j), 'IIYIYIXIII': (-0.007078805777596593+0j), 'IIXZXIXZXI': (0.011767712432203212+0j), 'IIXZXIYZYI': (0.011767712432203212+0j), 'IIYZYIXZXI': (0.011767712432203212+0j), 'IIYZYIYZYI': (0.011767712432203212+0j), 'ZIXXZZIZZI': (-0.0008617979837369933+0j), 'ZIYYZZIZIZ': (-0.0008617979837369933+0j), 'ZZXXIIIIZZ': (-0.0008617979837369933+0j), 'ZZYYIIIIII': (-0.0008617979837369933+0j), 'IIXYIIIIIY': (0.00021175546627918873+0j), 'IIYYIIIIIX': (-0.00021175546627918873+0j), 'ZIXYIZZIIY': (-0.00021175546627918873+0j), 'ZIYYIZZIIX': (0.00021175546627918873+0j), 'ZIYXIZZIYI': (0.0024136227932753133+0j), 'ZIYYIZZIXI': (-0.0024136227932753133+0j), 'ZZYXZIZZYZ': (0.0024136227932753133+0j), 'ZZYYZIZZXZ': (-0.0024136227932753133+0j), 'IIXXZIZXZZ': (0.007078805777596592-0j), 'IIXYZIZYII': (-0.007078805777596592+0j), 'IIYXZIZYZZ': (0.007078805777596592-0j), 'IIYYZIZXII': (0.007078805777596592+0j), 'IIXXIIIYYI': (-0.011030861199352059+0j), 'IIXYIIIYXI': (0.011030861199352059+0j), 'IIYXIIIXYI': (0.011030861199352059+0j), 'IIYYIIIXXI': (-0.011030861199352059+0j), 'IIXXIIXZZX': (-0.019337553877385297+0j), 'IIXYIIYZZX': (-0.019337553877385297+0j), 'IIYXIIXZZY': (-0.019337553877385297+0j), 'IIYYIIYZZY': (-0.019337553877385297+0j), 'IIXXXIIIZZ': (-0.011729648031149432+0j), 'IIXYYIIIII': (0.011729648031149432-0j), 'IIYXYIIIZZ': (-0.011729648031149432+0j), 'IIYYXIIIII': (-0.011729648031149432+0j), 'IIXXYIZZYI': (-0.0007271035496655186+0j), 'IIXYYIZZXI': (0.0007271035496655186-0j), 'IIYXXIZZYI': (0.0007271035496655186-0j), 'IIYYXIZZXI': (-0.0007271035496655186+0j), 'IXIIIIIIII': (0.003228125052947936+0j), 'IXIZZIZZII': (-0.10408986082348168+0j), 'IXZIZZIZIZ': (0.003228125052947936+0j), 'IXZZIZZIIZ': (-0.10408986082348168+0j), 'ZXIIIIIIII': (0.12506426575286977+0j), 'ZXIIIIIIIZ': (0.006942739716112276+0j), 'ZXIIIIIIZI': (0.0023629620252460528+0j), 'ZXIIIIIZII': (0.0033909458370214706+0j), 'ZXIIIIZIII': (0.0017393846301726602+0j), 'ZXIIIZIIII': (0.013952915826349686+0j), 'ZXIIZIIIII': (0.008168668890505323+0j), 'ZXIIZZIZIZ': (0.005549027597221014+0j), 'ZXIIZZIZZI': (0.0010997962401190324-0j), 'ZXIZIIIIII': (0.0034118780749390542+0j), 'ZXZIIIIIII': (0.005549027597221014+0j), 'ZXZIIIIIZZ': (0.0010997962401190324-0j), 'ZXZIIZIZIZ': (0.008168668890505323+0j), 'ZXZIZIIZIZ': (0.013952915826349686+0j), 'ZXZIZZIIIZ': (0.0033909458370214706+0j), 'ZXZIZZIZII': (0.006942739716112276+0j), 'ZXZIZZIZIZ': (0.12506426575286977+0j), 'ZXZIZZIZZZ': (0.0023629620252460528+0j), 'ZXZIZZZZIZ': (0.0017393846301726602+0j), 'ZXZZZZIZIZ': (0.0034118780749390542+0j), 'IXZIIIIIZX': (-0.0034628854945885513+0j), 'IYZIIIIIZY': (-0.0034628854945885513+0j), 'ZXZZIZZIIX': (-0.0034628854945885513+0j), 'ZYZZIZZIIY': (-0.0034628854945885513+0j), 'ZXIZIZZIXZ': (0.001881824516154209+0j), 'ZXIZZIZZXZ': (-0.001881824516154209+0j), 'ZXZZIZZIXI': (-0.001881824516154209+0j), 'ZXZZZIZZXI': (0.001881824516154209+0j), 'IXIZZIZZXX': (0.004579777690866224+0j), 'IYIZZIZZXY': (0.004579777690866224+0j), 'ZXZIZZIZYY': (-0.004579777690866224+0j), 'ZYZIZZIZYX': (0.004579777690866224+0j), 'IXIIZIZXZZ': (-0.022361664146433674+0j), 'IXIZIIZXZZ': (-0.02988890097022536+0j), 'IXIZZIIXZZ': (-0.024135168309130393+0j), 'IXIZZIZXIZ': (-0.017346179273540027+0j), 'IXIZZIZXZI': (-0.019844989410309235+0j), 'IXIZZIZXZZ': (-0.28534861549869345+0j), 'IXIZZZZXZZ': (-0.06469587851740755+0j), 'IXZZIZZXZI': (-0.06841162871444646+0j), 'IXZZZIZXII': (-0.018995063792228782+0j), 'IXZZZIZXZZ': (-0.029126564603127264+0j), 'IYIIZIZYZZ': (-0.022361664146433674+0j), 'IYIZIIZYZZ': (-0.02988890097022536+0j), 'IYIZZIIYZZ': (-0.024135168309130393+0j), 'IYIZZIZYIZ': (-0.017346179273540027+0j), 'IYIZZIZYZI': (-0.019844989410309235+0j), 'IYIZZIZYZZ': (-0.28534861549869345+0j), 'IYIZZZZYZZ': (-0.06469587851740755+0j), 'IYZZIZZYZI': (-0.06841162871444646+0j), 'IYZZZIZYII': (-0.018995063792228782+0j), 'IYZZZIZYZZ': (-0.029126564603127264+0j), 'ZXIZZIZXZZ': (-0.035773153993523474+0j), 'ZXZIZZIXZI': (0.07742200239782905+0j), 'ZYIZZIZYZZ': (-0.035773153993523474+0j), 'ZYZIZZIYZI': (0.07742200239782905+0j), 'IXIIIIIYYI': (0.019261279543166067-0j), 'IXZIIIIYYZ': (-0.019261279543166067+0j), 'IYIIIIIXYI': (-0.019261279543166067+0j), 'IYZIIIIXYZ': (0.019261279543166067+0j), 'IXIZZIXIZZ': (-0.0090103736833826+0j), 'IXZZIZXZZI': (-0.0090103736833826+0j), 'ZXIIIIXZZZ': (-0.0090103736833826+0j), 'ZXZIZZXIZI': (-0.0090103736833826+0j), 'IXIIIIXZZX': (0.008863394674723742-0j), 'IXZIIIYZIY': (0.008863394674723742+0j), 'IYIIIIXZZY': (0.008863394674723742-0j), 'IYZIIIYZIX': (-0.008863394674723742+0j), 'IXIZZIXIYY': (-0.0024988101367692107+0j), 'IXIZZIYIXY': (0.0024988101367692107+0j), 'IYIZZIXIYX': (0.0024988101367692107+0j), 'IYIZZIYIXX': (-0.0024988101367692107+0j), 'IXIZZIYYII': (-0.00165156120684881+0j), 'IYIZZIYXII': (0.00165156120684881+0j), 'ZXZIZZYYIZ': (-0.00165156120684881+0j), 'ZYZIZZYXIZ': (0.00165156120684881+0j), 'IXIZZXZZII': (-0.00811125152740662+0j), 'IYIZZYZZII': (-0.00811125152740662+0j), 'ZXZIZXIZIZ': (-0.00811125152740662+0j), 'ZYZIZYIZIZ': (-0.00811125152740662+0j), 'IXIZZXXIZZ': (0.014864979068627871-0j), 'IXIZZYYIZZ': (0.014864979068627871+0j), 'IYIZZXYIZZ': (-0.014864979068627871+0j), 'IYIZZYXIZZ': (0.014864979068627871-0j), 'IXIIXIIIZZ': (0.026160208111399748-0j), 'IXIZXIIIIZ': (0.0034451001404457804-0j), 'IXIZXIIIZI': (-0.0007287448167856668+0j), 'IXIZXIIIZZ': (0.11811415718604704-0j), 'IXIZXIIZZZ': (0.0004907767081770327-0j), 'IXIZXIZIZZ': (-0.014761267211071823+0j), 'IXIZXZIIZZ': (0.02232240891660917+0j), 'IXZZXIIIII': (-0.0017561216696856253+0j), 'IXZZXIIIZZ': (0.0031673513364151896-0j), 'IXZZXZIZZI': (0.026164329046178728-0j), 'IYIIYIIIZZ': (0.026160208111399748-0j), 'IYIZYIIIIZ': (0.0034451001404457804-0j), 'IYIZYIIIZI': (-0.0007287448167856668+0j), 'IYIZYIIIZZ': (0.11811415718604704-0j), 'IYIZYIIZZZ': (0.0004907767081770327-0j), 'IYIZYIZIZZ': (-0.014761267211071823+0j), 'IYIZYZIIZZ': (0.02232240891660917+0j), 'IYZZYIIIII': (-0.0017561216696856253+0j), 'IYZZYIIIZZ': (0.0031673513364151896-0j), 'IYZZYZIZZI': (0.026164329046178728-0j), 'ZXIZXIIIZZ': (-0.0012689361658123422+0j), 'ZXZIXZZIZI': (-0.03177067518883629+0j), 'ZYIZYIIIZZ': (-0.0012689361658123422+0j), 'ZYZIYZZIZI': (-0.03177067518883629+0j), 'IXIIYIZZYI': (-0.018950077979522674+0j), 'IXZIYIZZYZ': (0.018950077979522674-0j), 'IYIIXIZZYI': (0.018950077979522674+0j), 'IYZIXIZZYZ': (-0.018950077979522674+0j), 'ZXIIXIZXII': (-0.005610284205288328+0j), 'ZXIIYIZYII': (-0.005198675876396835+0j), 'ZXZIXZZXIZ': (0.005610284205288328-0j), 'ZXZIYZZYIZ': (0.005198675876396835-0j), 'ZYIIYIZXII': (-0.0004116083288914934+0j), 'ZYZIYZZXIZ': (0.0004116083288914934-0j), 'IXIZXIXZII': (-0.005070148182504693+0j), 'IYIZYIXZII': (-0.005070148182504693+0j), 'ZXZIXZXZIZ': (-0.005070148182504693+0j), 'ZYZIYZXZIZ': (-0.005070148182504693+0j), 'IXIXIIIIZZ': (0.005606346142657561-0j), 'IXZXZZIZZI': (0.005606346142657561-0j), 'ZXIXZIZZZZ': (0.005606346142657561-0j), 'ZXZXIZZIZI': (0.005606346142657561-0j), 'IXIXZIZZZX': (-0.008500365509750524+0j), 'IXZYZIZZIY': (-0.008500365509750524+0j), 'IYIXZIZZZY': (-0.008500365509750524+0j), 'IYZYZIZZIX': (0.008500365509750524-0j), 'IXIXIIIIYY': (-0.004173844957231447+0j), 'IXIYIIIIXY': (0.004173844957231447-0j), 'IYIXIIIIYX': (0.004173844957231447-0j), 'IYIYIIIIXX': (-0.004173844957231447+0j), 'IXIYIIIYII': (-0.004658539853613199+0j), 'IYIYIIIXII': (0.004658539853613199-0j), 'ZXZYIZZYIZ': (-0.004658539853613199+0j), 'ZYZYIZZXIZ': (0.004658539853613199-0j), 'ZXIXZIXIII': (-0.0005401360227836365+0j), 'ZXIYZIYIII': (-0.0005401360227836365+0j), 'ZXZXIZXZIZ': (-0.0005401360227836365+0j), 'ZXZYIZYZIZ': (-0.0005401360227836365+0j), 'IXIXIIXXZZ': (-0.015252043919248854+0j), 'IXIYIIYXZZ': (-0.015252043919248854+0j), 'IYIXIIXYZZ': (-0.015252043919248854+0j), 'IYIYIIYYZZ': (-0.015252043919248854+0j), 'IXIXIXIIZZ': (-0.011704287187750151+0j), 'IXIYIYIIZZ': (-0.011704287187750151+0j), 'IYIXIYIIZZ': (-0.011704287187750151+0j), 'IYIYIXIIZZ': (0.011704287187750151-0j), 'IXIYYIZZII': (-0.0047567908155662685+0j), 'IYIYXIZZII': (0.0047567908155662685+0j), 'ZXZYYZIZIZ': (-0.0047567908155662685+0j), 'ZYZYXZIZIZ': (0.0047567908155662685+0j), 'IXIXYIYIZZ': (0.007527236823791688+0j), 'IXIYYIXIZZ': (-0.007527236823791688+0j), 'IYIXXIYIZZ': (-0.007527236823791688+0j), 'IYIYXIXIZZ': (0.007527236823791688+0j), 'IXXZZIZZII': (-0.004449231357101981+0j), 'IYYZZIZZII': (-0.004449231357101981+0j), 'ZXXIZZIZZI': (-0.004449231357101981+0j), 'ZYYIZZIZZI': (-0.004449231357101981+0j), 'ZXXZIZZIZX': (0.00767996475591476-0j), 'ZXXZZIZZZX': (-0.00767996475591476+0j), 'ZXYZIZZIZY': (0.005344710010742759-0j), 'ZXYZZIZZZY': (-0.005344710010742759+0j), 'ZYYZIZZIZX': (0.0023352547451720006-0j), 'ZYYZZIZZZX': (-0.0023352547451720006+0j), 'IXYIIIIIYI': (0.005798140239760552-0j), 'IYXIIIIIYI': (-0.005798140239760552+0j), 'ZXXZIZZIXZ': (-0.005798140239760552+0j), 'ZYYZIZZIXZ': (-0.005798140239760552+0j), 'IXXIIIIYIY': (0.010397884868442325-0j), 'IXYIIIIXIY': (-0.01593846664015537+0j), 'IXYIIIIYIX': (0.005540581771713046-0j), 'IYXIIIIXIY': (0.005540581771713046-0j), 'IYXIIIIYIX': (-0.01593846664015537+0j), 'IYYIIIIXIX': (0.010397884868442325-0j), 'IXXZZIXIZZ': (-0.010131500810898484+0j), 'IXYZZIYIII': (-0.010131500810898484+0j), 'IYXZZIYIII': (0.010131500810898484+0j), 'IYYZZIXIZZ': (-0.010131500810898484+0j), 'IXYIIIXZYZ': (-0.024801861314879114+0j), 'IXYIIIYZXZ': (0.024801861314879114-0j), 'IYXIIIXZYZ': (0.024801861314879114-0j), 'IYXIIIYZXZ': (-0.024801861314879114+0j), 'IXXIYIZZIY': (-0.01044971246977215+0j), 'IXYIXIZZIY': (0.01075145073809608+0j), 'IXYIYIZZIX': (-0.0003017382683239327+0j), 'IYXIXIZZIY': (-0.0003017382683239327+0j), 'IYXIYIZZIX': (0.01075145073809608+0j), 'IYYIXIZZIX': (-0.01044971246977215+0j), 'IXXXIIIIZZ': (0.004923473006100815+0j), 'IXYYIIIIII': (0.004923473006100815-0j), 'IYXYIIIIII': (-0.004923473006100815+0j), 'IYYXIIIIZZ': (0.004923473006100815+0j), 'IXYXZIZZYZ': (0.019251816247846606+0j), 'IXYYZIZZXZ': (-0.019251816247846606+0j), 'IYXXZIZZYZ': (-0.019251816247846606+0j), 'IYXYZIZZXZ': (0.019251816247846606+0j), 'XIIIIIIIII': (0.10408986082348168+0j), 'XIIZZIZZII': (-0.003228125052947936+0j), 'XIZIZZIZIZ': (-0.003228125052947936+0j), 'XIZZIZZIIZ': (0.10408986082348168+0j), 'XZIIZIZZII': (-0.008168668890505323+0j), 'XZIIZZIZIZ': (-0.0010997962401190324+0j), 'XZIIZZIZZI': (-0.005549027597221014+0j), 'XZIZIIZZII': (-0.0034118780749390542+0j), 'XZIZZIIZII': (-0.0033909458370214706+0j), 'XZIZZIZIII': (-0.0017393846301726602+0j), 'XZIZZIZZII': (-0.12506426575286975+0j), 'XZIZZIZZIZ': (-0.0023629620252460528+0j), 'XZIZZIZZZI': (-0.006942739716112276+0j), 'XZIZZZZZII': (-0.013952915826349686+0j), 'XZZIIZIZIZ': (-0.0034118780749390542+0j), 'XZZIZIIZIZ': (-0.013952915826349686+0j), 'XZZIZZIIIZ': (-0.0017393846301726602+0j), 'XZZIZZIZII': (-0.0023629620252460528+0j), 'XZZIZZIZIZ': (-0.12506426575286975+0j), 'XZZIZZIZZZ': (-0.006942739716112276+0j), 'XZZIZZZZIZ': (-0.0033909458370214706+0j), 'XZZZZIZZII': (-0.0010997962401190324+0j), 'XZZZZIZZZZ': (-0.005549027597221014+0j), 'XZZZZZIZIZ': (-0.008168668890505323+0j), 'XIZZZIZZZX': (0.005798140239760552+0j), 'XZIZIZZIZX': (0.005798140239760552+0j), 'YIIZZIZZIY': (0.005798140239760552-0j), 'YZZZIZZIIY': (0.005798140239760552-0j), 'XZIIIIIIXZ': (0.00767996475591476-0j), 'XZIZIZZIXZ': (-0.00767996475591476+0j), 'XZZIIIIIXI': (-0.005344710010742759+0j), 'XZZZIZZIXI': (0.005344710010742759-0j), 'YZZIIIIIYI': (0.0023352547451720006+0j), 'YZZZIZZIYI': (-0.0023352547451720006+0j), 'XIIIIIIIXX': (-0.004579777690866223+0j), 'XZZIZZIZXX': (-0.004579777690866223+0j), 'YIIIIIIIYX': (-0.004579777690866223+0j), 'YZZIZZIZYX': (-0.004579777690866223+0j), 'YIIIIIIYZZ': (-0.009010373683382603+0j), 'YIZZIZZYZI': (-0.009010373683382603+0j), 'YZIZZIZYZZ': (-0.009010373683382603+0j), 'YZZIZZIYZI': (-0.009010373683382603+0j), 'XZIIIIIXZX': (-0.024801861314879114+0j), 'XZIIIIIYZY': (-0.024801861314879114+0j), 'YZZIIIIXIY': (-0.024801861314879114+0j), 'YZZIIIIYIX': (0.024801861314879114+0j), 'XZIZZIZXXX': (-0.0024988101367692103+0j), 'XZIZZIZYXY': (-0.0024988101367692103+0j), 'YZIZZIZXYX': (-0.0024988101367692103+0j), 'YZIZZIZYYY': (-0.0024988101367692103+0j), 'XIIZZIXIZZ': (-0.035773153993523474+0j), 'XIZZIZXZZI': (0.07742200239782905+0j), 'XZIIZIXIZZ': (-0.02988890097022536+0j), 'XZIZIIXIZZ': (-0.022361664146433674+0j), 'XZIZZIXIIZ': (-0.019844989410309235+0j), 'XZIZZIXIZI': (-0.017346179273540027+0j), 'XZIZZIXIZZ': (-0.28534861549869345+0j), 'XZIZZIXZZZ': (-0.024135168309130393+0j), 'XZIZZZXIZZ': (-0.06469587851740755+0j), 'XZZIZZXIZI': (-0.06841162871444646+0j), 'XZZZZIXIII': (-0.029126564603127264+0j), 'XZZZZIXIZZ': (-0.018995063792228782+0j), 'YIIZZIYIZZ': (-0.035773153993523474+0j), 'YIZZIZYZZI': (0.07742200239782905+0j), 'YZIIZIYIZZ': (-0.02988890097022536+0j), 'YZIZIIYIZZ': (-0.022361664146433674+0j), 'YZIZZIYIIZ': (-0.019844989410309235+0j), 'YZIZZIYIZI': (-0.017346179273540027+0j), 'YZIZZIYIZZ': (-0.28534861549869345+0j), 'YZIZZIYZZZ': (-0.024135168309130393+0j), 'YZIZZZYIZZ': (-0.06469587851740755+0j), 'YZZIZZYIZI': (-0.06841162871444646+0j), 'YZZZZIYIII': (-0.029126564603127264+0j), 'YZZZZIYIZZ': (-0.018995063792228782+0j), 'XZIIIIXZXI': (-0.01593846664015537+0j), 'XZIIIIYZYI': (-0.005540581771713046+0j), 'XZZIIIYZYZ': (-0.010397884868442325+0j), 'YZIIIIXZYI': (-0.010397884868442325+0j), 'YZZIIIXZYZ': (-0.005540581771713046+0j), 'YZZIIIYZXZ': (0.01593846664015537+0j), 'XIIIIIXXII': (-0.0016515612068488097+0j), 'XZZIZZXXIZ': (-0.0016515612068488097+0j), 'YIIIIIYXII': (-0.0016515612068488097+0j), 'YZZIZZYXIZ': (-0.0016515612068488097+0j), 'XIIIIXIIII': (0.00811125152740662-0j), 'XZZIZXIZIZ': (0.00811125152740662-0j), 'YIIIIYIIII': (0.00811125152740662+0j), 'YZZIZYIZIZ': (0.00811125152740662+0j), 'XZIZZXZXZZ': (-0.01486497906862787+0j), 'XZIZZYZYZZ': (-0.01486497906862787+0j), 'YZIZZXZYZZ': (0.01486497906862787-0j), 'YZIZZYZXZZ': (-0.01486497906862787+0j), 'YIIIYIZZZZ': (0.00560634614265756-0j), 'YIZZYZIZZI': (0.00560634614265756-0j), 'YZIZYIIIZZ': (0.00560634614265756-0j), 'YZZIYZZIZI': (0.00560634614265756-0j), 'XZIIXIZZZX': (0.019251816247846606+0j), 'XZIIYIZZZY': (0.019251816247846606+0j), 'YZZIXIZZIY': (0.019251816247846606-0j), 'YZZIYIZZIX': (-0.019251816247846606+0j), 'XZIZXIIIXX': (-0.004173844957231447+0j), 'XZIZYIIIXY': (-0.004173844957231447+0j), 'YZIZXIIIYX': (-0.004173844957231447+0j), 'YZIZYIIIYY': (-0.004173844957231447+0j), 'XZIZXIIXII': (-0.0005401360227836365+0j), 'XZIZYIIYII': (-0.0005401360227836365+0j), 'XZZIXZZXIZ': (-0.0005401360227836365+0j), 'XZZIYZZYIZ': (-0.0005401360227836365+0j), 'XIIIYIYIII': (-0.004658539853613199+0j), 'XZZIYZYZIZ': (-0.004658539853613199+0j), 'YIIIYIXIII': (0.004658539853613199-0j), 'YZZIYZXZIZ': (0.004658539853613199-0j), 'XZIZXIYYZZ': (-0.015252043919248854+0j), 'XZIZYIYXZZ': (0.015252043919248854-0j), 'YZIZXIXYZZ': (0.015252043919248854-0j), 'YZIZYIXXZZ': (-0.015252043919248854+0j), 'XZIZXXIIZZ': (0.01170428718775015-0j), 'XZIZYYIIZZ': (0.01170428718775015-0j), 'YZIZXYIIZZ': (0.01170428718775015+0j), 'YZIZYXIIZZ': (-0.01170428718775015+0j), 'XIIXIIIIZZ': (-0.0012689361658123422+0j), 'XIZXZZIZZI': (-0.03177067518883629+0j), 'XZIXIIIIIZ': (-0.0007287448167856668+0j), 'XZIXIIIIZI': (0.0034451001404457804-0j), 'XZIXIIIIZZ': (0.11811415718604702-0j), 'XZIXIIIZZZ': (-0.014761267211071823+0j), 'XZIXIIZIZZ': (0.0004907767081770327-0j), 'XZIXIZIIZZ': (0.02232240891660917+0j), 'XZIXZIIIZZ': (0.026160208111399748-0j), 'XZZXIIIIII': (0.0031673513364151896+0j), 'XZZXIIIIZZ': (-0.0017561216696856253+0j), 'XZZXIZZIZI': (0.026164329046178728-0j), 'YIIYIIIIZZ': (-0.0012689361658123422+0j), 'YIZYZZIZZI': (-0.03177067518883629+0j), 'YZIYIIIIIZ': (-0.0007287448167856668+0j), 'YZIYIIIIZI': (0.0034451001404457804-0j), 'YZIYIIIIZZ': (0.11811415718604702-0j), 'YZIYIIIZZZ': (-0.014761267211071823+0j), 'YZIYIIZIZZ': (0.0004907767081770327-0j), 'YZIYIZIIZZ': (0.02232240891660917+0j), 'YZIYZIIIZZ': (0.026160208111399748-0j), 'YZZYIIIIII': (0.0031673513364151896+0j), 'YZZYIIIIZZ': (-0.0017561216696856253+0j), 'YZZYIZZIZI': (0.026164329046178728-0j), 'XZIXZIZZXI': (0.01075145073809608+0j), 'XZIYZIZZYI': (0.0003017382683239327+0j), 'XZZYZIZZYZ': (0.01044971246977215-0j), 'YZIXZIZZYI': (0.01044971246977215+0j), 'YZZXZIZZYZ': (0.0003017382683239327-0j), 'YZZYZIZZXZ': (-0.01075145073809608+0j), 'XIIXZIZXII': (-0.005070148182504693+0j), 'XZZXIZZXIZ': (-0.005070148182504693+0j), 'YIIYZIZXII': (-0.005070148182504693+0j), 'YZZYIZZXIZ': (-0.005070148182504693+0j), 'XZIXIIXZII': (-0.005610284205288328+0j), 'XZIYIIYZII': (-0.005198675876396835+0j), 'XZZXIZXZIZ': (0.005610284205288328-0j), 'XZZYIZYZIZ': (0.005198675876396835-0j), 'YZIYIIXZII': (-0.0004116083288914934+0j), 'YZZYIZXZIZ': (0.0004116083288914934-0j), 'XIIXXIIIII': (-0.0047567908155662685+0j), 'XZZXXZIZIZ': (-0.0047567908155662685+0j), 'YIIYXIIIII': (-0.0047567908155662685+0j), 'YZZYXZIZIZ': (-0.0047567908155662685+0j), 'XZIXXIZXZZ': (-0.007527236823791688+0j), 'XZIXYIZYZZ': (-0.007527236823791688+0j), 'YZIYXIZXZZ': (-0.007527236823791688+0j), 'YZIYYIZYZZ': (-0.007527236823791688+0j), 'XIXIIIIIII': (0.004449231357101981-0j), 'XZXIZZIZIZ': (0.004449231357101981-0j), 'YIYIIIIIZZ': (0.004449231357101981+0j), 'YZYIZZIZZI': (0.004449231357101981+0j), 'XZXIIIIIZX': (0.001881824516154209-0j), 'XZXZIZZIZX': (-0.001881824516154209+0j), 'XZYIIIIIZY': (0.001881824516154209-0j), 'XZYZIZZIZY': (-0.001881824516154209+0j), 'XIYZZIZZYI': (-0.0034628854945885513+0j), 'XZYZIZZIYZ': (-0.0034628854945885513+0j), 'YIYZZIZZXI': (0.0034628854945885513-0j), 'YZYZIZZIXZ': (0.0034628854945885513-0j), 'XZXZZIZXII': (0.010131500810898484+0j), 'XZYZZIZYII': (0.010131500810898484+0j), 'YZXZZIZYZZ': (-0.010131500810898484+0j), 'YZYZZIZXZZ': (0.010131500810898484-0j), 'XZXIIIIYYZ': (-0.008863394674723742+0j), 'XZYIIIIXYZ': (0.008863394674723742-0j), 'YZXIIIIYXZ': (0.008863394674723742-0j), 'YZYIIIIXXZ': (-0.008863394674723742+0j), 'XZXIIIYZIY': (0.019261279543166067-0j), 'XZYIIIYZIX': (-0.019261279543166067+0j), 'YZXIIIXZIY': (-0.019261279543166067+0j), 'YZYIIIXZIX': (0.019261279543166067-0j), 'XZXZXIIIII': (-0.004923473006100815+0j), 'XZYZYIIIII': (-0.004923473006100815+0j), 'YZXZYIIIZZ': (0.004923473006100815+0j), 'YZYZXIIIZZ': (-0.004923473006100815+0j), 'XZXIYIZZYZ': (0.008500365509750524+0j), 'XZYIXIZZYZ': (-0.008500365509750524+0j), 'YZXIYIZZXZ': (-0.008500365509750524+0j), 'YZYIXIZZXZ': (0.008500365509750524+0j), 'XZXYZIZZIY': (-0.018950077979522674+0j), 'XZYYZIZZIX': (0.018950077979522674+0j), 'YZXXZIZZIY': (0.018950077979522674+0j), 'YZYXZIZZIX': (-0.018950077979522674+0j), 'YYIIIIIIII': (-0.014526685537013806+0j), 'YYIZZIZZII': (-0.014526685537013806+0j), 'YYZIZZIZIZ': (-0.014526685537013806+0j), 'YYZZIZZIIZ': (-0.014526685537013806+0j), 'XXIZZIZZIX': (-0.00998211145829183+0j), 'XYIZZIZZIY': (-0.00998211145829183+0j), 'YXZZZIZZZY': (-0.00998211145829183+0j), 'YYZZZIZZZX': (0.00998211145829183+0j), 'XXIIIIIIYY': (-0.015430252188811678+0j), 'XYIIIIIIYX': (0.015430252188811678+0j), 'YXIIIIIIXY': (0.015430252188811678+0j), 'YYIIIIIIXX': (-0.015430252188811678+0j), 'XXIIIIIXZZ': (-0.0016731487702616589+0j), 'XXZZIZZXZI': (-0.0016731487702616589+0j), 'XYIIIIIYZZ': (-0.0016731487702616589+0j), 'XYZZIZZYZI': (-0.0016731487702616589+0j), 'XYIZZIYIZZ': (0.0016731487702616589+0j), 'XYZZIZYZZI': (0.0016731487702616589+0j), 'YYIZZIXIZZ': (-0.0016731487702616589+0j), 'YYZZIZXZZI': (-0.0016731487702616589+0j), 'XXIIIIYYII': (-0.025510064011573047+0j), 'XYIIIIYXII': (0.025510064011573047+0j), 'YXIIIIXYII': (0.025510064011573047+0j), 'YYIIIIXXII': (-0.025510064011573047+0j), 'XXIIIXIIII': (-0.036081827421618036+0j), 'XYIIIYIIII': (-0.036081827421618036+0j), 'YXIIIYIIII': (-0.036081827421618036+0j), 'YYIIIXIIII': (0.036081827421618036-0j), 'XXIIXIZZZZ': (0.002306121500740196-0j), 'XXZZXZIZZI': (0.002306121500740196-0j), 'XYIIYIZZZZ': (0.002306121500740196-0j), 'XYZZYZIZZI': (0.002306121500740196-0j), 'XXIIXIXIII': (-0.005188080440041431+0j), 'XYIIYIXIII': (-0.005188080440041431+0j), 'YXIIXIYIII': (-0.005188080440041431+0j), 'YYIIYIYIII': (-0.005188080440041431+0j), 'XYIYIIIIZZ': (-0.0023061215007401962+0j), 'XYZYZZIZZI': (-0.0023061215007401962+0j), 'YYIXIIIIZZ': (0.0023061215007401962-0j), 'YYZXZZIZZI': (0.0023061215007401962-0j), 'XXIYZIZYII': (-0.005188080440041431+0j), 'XYIYZIZXII': (0.005188080440041431-0j), 'YXIXZIZYII': (0.005188080440041431-0j), 'YYIXZIZXII': (-0.005188080440041431+0j), 'XXIYYIIIII': (-0.030954328800404832+0j), 'XYIYXIIIII': (0.030954328800404832+0j), 'YXIXYIIIII': (0.030954328800404832+0j), 'YYIXXIIIII': (-0.030954328800404832+0j), 'XXXIIIIIZZ': (-0.03623664575511546+0j), 'XYYIIIIIZZ': (-0.03623664575511546+0j), 'YXYIIIIIII': (-0.03623664575511546+0j), 'YYXIIIIIII': (0.03623664575511546-0j), 'XXXZZIZZXI': (-0.00998211145829183+0j), 'XYYZZIZZXI': (-0.00998211145829183+0j), 'YXXZZIZZYI': (-0.00998211145829183+0j), 'YYYZZIZZYI': (-0.00998211145829183+0j)}
H2O_ucc_dict = {'IIIZZIZZYZ': 0.0016283902641745136j, 'IIZZZIZZYI': -0.0016283902641745136j, 'IIIIIIIIXY': 5.3687492916235514e-05j, 'IIIIIIIIYX': 5.3687492916235514e-05j, 'IZZIZZIZXY': 5.3687492916235514e-05j, 'IZZIZZIZYX': 5.3687492916235514e-05j, 'ZIZZIZZIXY': 5.3687492916235514e-05j, 'ZIZZIZZIYX': 5.3687492916235514e-05j, 'ZZIZZIZZXY': 5.3687492916235514e-05j, 'ZZIZZIZZYX': 5.3687492916235514e-05j, 'ZIZZIZZYZI': 1.5263860793850173e-05j, 'ZZIZZIZYZZ': 1.5263860793850173e-05j, 'ZIIIZZIXYI': -3.040341974887387e-05j, 'ZIIIZZIYXI': -3.040341974887387e-05j, 'ZIZIZZIXYZ': 3.040341974887387e-05j, 'ZIZIZZIYXZ': 3.040341974887387e-05j, 'ZZIIIIIXYI': 3.040341974887387e-05j, 'ZZIIIIIYXI': 3.040341974887387e-05j, 'ZZZIIIIXYZ': -3.040341974887387e-05j, 'ZZZIIIIYXZ': -3.040341974887387e-05j, 'IIIIIIYZZZ': -1.5263860793850173e-05j, 'ZIZZIZYZZI': -1.5263860793850173e-05j, 'ZIIIZZXIIY': -3.040341974887387e-05j, 'ZIIIZZYIIX': -3.040341974887387e-05j, 'ZIZIZZXIZY': 3.040341974887387e-05j, 'ZIZIZZYIZX': 3.040341974887387e-05j, 'ZZIIIIXZZY': 3.040341974887387e-05j, 'ZZIIIIYZZX': 3.040341974887387e-05j, 'ZZZIIIXZIY': -3.040341974887387e-05j, 'ZZZIIIYZIX': -3.040341974887387e-05j, 'IIIZZIXIYI': (-0-6.080683949774774e-05j), 'IIIZZIYIXI': (-0-6.080683949774774e-05j), 'IIZZZIXIYZ': 6.080683949774774e-05j, 'IIZZZIYIXZ': 6.080683949774774e-05j, 'ZIIIZZXIYI': 6.080683949774774e-05j, 'ZIIIZZYIXI': 6.080683949774774e-05j, 'ZIZIZZXIYZ': (-0-6.080683949774774e-05j), 'ZIZIZZYIXZ': (-0-6.080683949774774e-05j), 'IIIIIIXYII': 8.194039564011285e-05j, 'IIIIIIYXII': 8.194039564011285e-05j, 'IZZIZZXYIZ': 8.194039564011285e-05j, 'IZZIZZYXIZ': 8.194039564011285e-05j, 'ZIZZIZXYIZ': 8.194039564011285e-05j, 'ZIZZIZYXIZ': 8.194039564011285e-05j, 'ZZIZZIXYII': 8.194039564011285e-05j, 'ZZIZZIYXII': 8.194039564011285e-05j, 'IIIIIXIIXY': -0.0026363068445567203j, 'IIIIIXIIYX': -0.0026363068445567203j, 'IIIIIYIIXX': 0.0026363068445567203j, 'IIIIIYIIYY': -0.0026363068445567203j, 'IIIIIXXYII': -0.006532364829996931j, 'IIIIIXYXII': -0.006532364829996931j, 'IIIIIYXXII': 0.006532364829996931j, 'IIIIIYYYII': -0.006532364829996931j, 'IIIIXIZYII': 0.006547703771488601j, 'IIIIYIZXII': (-0-0.006547703771488601j), 'IIIZXIIXYZ': -0.0007611919158043911j, 'IIIZXIIYXZ': -0.0007611919158043911j, 'IIIZYIIXXZ': 0.0007611919158043911j, 'IIIZYIIYYZ': -0.0007611919158043911j, 'IIZZXIIXYI': 0.0007611919158043911j, 'IIZZXIIYXI': 0.0007611919158043911j, 'IIZZYIIXXI': (-0-0.0007611919158043911j), 'IIZZYIIYYI': 0.0007611919158043911j, 'IIIZXIXZIY': -0.0007611919158043911j, 'IIIZXIYZIX': -0.0007611919158043911j, 'IIIZYIXZIX': 0.0007611919158043911j, 'IIIZYIYZIY': -0.0007611919158043911j, 'IIZZXIXZZY': 0.0007611919158043911j, 'IIZZXIYZZX': 0.0007611919158043911j, 'IIZZYIXZZX': (-0-0.0007611919158043911j), 'IIZZYIYZZY': 0.0007611919158043911j, 'IIIXZIYIII': 0.006547703771488601j, 'IIIYZIXIII': (-0-0.006547703771488601j), 'IIIXIIXZYZ': -0.0015223838316087823j, 'IIIXIIYZXZ': -0.0015223838316087823j, 'IIIYIIXZXZ': 0.0015223838316087823j, 'IIIYIIYZYZ': -0.0015223838316087823j, 'IIZXIIXZYI': 0.0015223838316087823j, 'IIZXIIYZXI': 0.0015223838316087823j, 'IIZYIIXZXI': (-0-0.0015223838316087823j), 'IIZYIIYZYI': 0.0015223838316087823j, 'IIIXXIIIXY': 0.005180710419573264j, 'IIIXXIIIYX': 0.005180710419573264j, 'IIIXYIIIXX': -0.005180710419573264j, 'IIIXYIIIYY': 0.005180710419573264j, 'IIIYXIIIXX': -0.005180710419573264j, 'IIIYXIIIYY': 0.005180710419573264j, 'IIIYYIIIXY': -0.005180710419573264j, 'IIIYYIIIYX': -0.005180710419573264j, 'IIIXXIXYII': 0.005676861671006199j, 'IIIXXIYXII': 0.005676861671006199j, 'IIIXYIXXII': -0.005676861671006199j, 'IIIXYIYYII': 0.005676861671006199j, 'IIIYXIXXII': -0.005676861671006199j, 'IIIYXIYYII': 0.005676861671006199j, 'IIIYYIXYII': -0.005676861671006199j, 'IIIYYIYXII': -0.005676861671006199j, 'IIXZZIZZZY': 0.0016283902641745136j, 'IIYZZIZZZX': (-0-0.0016283902641745136j), 'IIXIIIIIXY': -0.019277850217549214j, 'IIXIIIIIYX': -0.019277850217549214j, 'IIYIIIIIXX': 0.019277850217549214j, 'IIYIIIIIYY': -0.019277850217549214j, 'ZIXIZZIXIY': (-0-6.080683949774774e-05j), 'ZIXIZZIYIX': (-0-6.080683949774774e-05j), 'ZIYIZZIXIX': 6.080683949774774e-05j, 'ZIYIZZIYIY': (-0-6.080683949774774e-05j), 'ZZXIIIIXIY': 6.080683949774774e-05j, 'ZZXIIIIYIX': 6.080683949774774e-05j, 'ZZYIIIIXIX': (-0-6.080683949774774e-05j), 'ZZYIIIIYIY': 6.080683949774774e-05j, 'IIXZZIZXYZ': (-0-3.040341974887387e-05j), 'IIXZZIZYXZ': (-0-3.040341974887387e-05j), 'IIYZZIZXXZ': 3.040341974887387e-05j, 'IIYZZIZYYZ': (-0-3.040341974887387e-05j), 'ZIXIZZIXYI': 3.040341974887387e-05j, 'ZIXIZZIYXI': 3.040341974887387e-05j, 'ZIYIZZIXXI': (-0-3.040341974887387e-05j), 'ZIYIZZIYYI': 3.040341974887387e-05j, 'IIXZZIXIIY': (-0-3.040341974887387e-05j), 'IIXZZIYIIX': (-0-3.040341974887387e-05j), 'IIYZZIXIIX': 3.040341974887387e-05j, 'IIYZZIYIIY': (-0-3.040341974887387e-05j), 'ZIXIZZXIIY': 3.040341974887387e-05j, 'ZIXIZZYIIX': 3.040341974887387e-05j, 'ZIYIZZXIIX': (-0-3.040341974887387e-05j), 'ZIYIZZYIIY': 3.040341974887387e-05j, 'IIXIIIXYII': (-0-0.005404441907474658j), 'IIXIIIXYZZ': -0.005404441907474658j, 'IIXIIIYXII': (-0-0.005404441907474658j), 'IIXIIIYXZZ': -0.005404441907474658j, 'IIYIIIXXII': 0.005404441907474658j, 'IIYIIIXXZZ': 0.005404441907474658j, 'IIYIIIYYII': (-0-0.005404441907474658j), 'IIYIIIYYZZ': -0.005404441907474658j, 'IIXZXIIXZY': -0.0015223838316087823j, 'IIXZXIIYZX': -0.0015223838316087823j, 'IIXZYIIXZX': 0.0015223838316087823j, 'IIXZYIIYZY': -0.0015223838316087823j, 'IIYZXIIXZX': 0.0015223838316087823j, 'IIYZXIIYZY': -0.0015223838316087823j, 'IIYZYIIXZY': 0.0015223838316087823j, 'IIYZYIIYZX': 0.0015223838316087823j, 'IIXXIIIXYI': -0.0007611919158043911j, 'IIXXIIIYXI': -0.0007611919158043911j, 'IIXYIIIXXI': 0.0007611919158043911j, 'IIXYIIIYYI': -0.0007611919158043911j, 'IIYXIIIXXI': 0.0007611919158043911j, 'IIYXIIIYYI': -0.0007611919158043911j, 'IIYYIIIXYI': 0.0007611919158043911j, 'IIYYIIIYXI': 0.0007611919158043911j, 'IIXXIIXZZY': -0.0007611919158043911j, 'IIXXIIYZZX': -0.0007611919158043911j, 'IIXYIIXZZX': 0.0007611919158043911j, 'IIXYIIYZZY': -0.0007611919158043911j, 'IIYXIIXZZX': 0.0007611919158043911j, 'IIYXIIYZZY': -0.0007611919158043911j, 'IIYYIIXZZY': 0.0007611919158043911j, 'IIYYIIYZZX': 0.0007611919158043911j, 'IXIZZIZYZZ': 0.00015036837398579362j, 'IYIZZIZXZZ': -0.00015036837398579362j, 'IXIIIIIXYI': (-0-0.0006795572380520797j), 'IXIIIIIYXI': (-0-0.0006795572380520797j), 'IXZIIIIXYZ': 0.0006795572380520797j, 'IXZIIIIYXZ': 0.0006795572380520797j, 'IYIIIIIXXI': 0.0006795572380520797j, 'IYIIIIIYYI': (-0-0.0006795572380520797j), 'IYZIIIIXXZ': -0.0006795572380520797j, 'IYZIIIIYYZ': 0.0006795572380520797j, 'IXIIIIXZZY': (-0-0.0006795572380520797j), 'IXIIIIYZZX': (-0-0.0006795572380520797j), 'IXZIIIXZIY': 0.0006795572380520797j, 'IXZIIIYZIX': 0.0006795572380520797j, 'IYIIIIXZZX': 0.0006795572380520797j, 'IYIIIIYZZY': (-0-0.0006795572380520797j), 'IYZIIIXZIX': -0.0006795572380520797j, 'IYZIIIYZIY': 0.0006795572380520797j, 'IXXIIIIXIY': (-0-0.0013591144761041594j), 'IXXIIIIYIX': (-0-0.0013591144761041594j), 'IXYIIIIXIX': 0.0013591144761041594j, 'IXYIIIIYIY': (-0-0.0013591144761041594j), 'IYXIIIIXIX': 0.0013591144761041594j, 'IYXIIIIYIY': (-0-0.0013591144761041594j), 'IYYIIIIXIY': 0.0013591144761041594j, 'IYYIIIIYIX': 0.0013591144761041594j, 'XZIZZIYIZZ': 0.00015036837398579362j, 'YZIZZIXIZZ': -0.00015036837398579362j, 'XZIIIIXZYI': (-0-0.0013591144761041594j), 'XZIIIIYZXI': (-0-0.0013591144761041594j), 'XZZIIIXZYZ': 0.0013591144761041594j, 'XZZIIIYZXZ': 0.0013591144761041594j, 'YZIIIIXZXI': 0.0013591144761041594j, 'YZIIIIYZYI': (-0-0.0013591144761041594j), 'YZZIIIXZXZ': -0.0013591144761041594j, 'YZZIIIYZYZ': 0.0013591144761041594j, 'XZXIIIIXYZ': (-0-0.0006795572380520797j), 'XZXIIIIYXZ': (-0-0.0006795572380520797j), 'XZYIIIIXXZ': 0.0006795572380520797j, 'XZYIIIIYYZ': (-0-0.0006795572380520797j), 'YZXIIIIXXZ': 0.0006795572380520797j, 'YZXIIIIYYZ': (-0-0.0006795572380520797j), 'YZYIIIIXYZ': 0.0006795572380520797j, 'YZYIIIIYXZ': 0.0006795572380520797j, 'XZXIIIXZIY': (-0-0.0006795572380520797j), 'XZXIIIYZIX': (-0-0.0006795572380520797j), 'XZYIIIXZIX': 0.0006795572380520797j, 'XZYIIIYZIY': (-0-0.0006795572380520797j), 'YZXIIIXZIX': 0.0006795572380520797j, 'YZXIIIYZIY': (-0-0.0006795572380520797j), 'YZYIIIXZIY': 0.0006795572380520797j, 'YZYIIIYZIX': 0.0006795572380520797j, 'XXIIIIIIXY': 0.002066601301708942j, 'XXIIIIIIYX': 0.002066601301708942j, 'XYIIIIIIXX': -0.002066601301708942j, 'XYIIIIIIYY': 0.002066601301708942j, 'YXIIIIIIXX': -0.002066601301708942j, 'YXIIIIIIYY': 0.002066601301708942j, 'YYIIIIIIXY': -0.002066601301708942j, 'YYIIIIIIYX': -0.002066601301708942j, 'XXIIIIXYII': 0.004041823552427175j, 'XXIIIIYXII': 0.004041823552427175j, 'XYIIIIXXII': -0.004041823552427175j, 'XYIIIIYYII': 0.004041823552427175j, 'YXIIIIXXII': -0.004041823552427175j, 'YXIIIIYYII': 0.004041823552427175j, 'YYIIIIXYII': -0.004041823552427175j, 'YYIIIIYXII': -0.004041823552427175j}
H2O_ref_state = np.array([1, 1, 1, 1, 1, 1, 0, 0, 0, 0])
H2O_fci_energy = -75.01084664816945

ham = PauliwordOp(H2O_ham_dict)
ucc = PauliwordOp(H2O_ucc_dict)
ref = H2O_ref_state

ham = ObservableOp(ham.symp_matrix, ham.coeff_vec.real)
ucc = AnsatzOp(ucc.symp_matrix, ucc.coeff_vec.imag)

In [17]:
help(ham)#.rotate_by_single_Pword

Help on ObservableOp in module symred.symplectic_form object:

class ObservableOp(PauliwordOp)
 |  Based on PauliwordOp and introduces functionality for evaluating expectation values
 |  
 |  Method resolution order:
 |      ObservableOp
 |      PauliwordOp
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  VQE(self, ansatz_op:symred.symplectic_form.AnsatzOp, ref_state:<built-in function array>, optimizer:str='SLSQP', maxiter:int=10, opt_tol:float=None)
 |  
 |  Z_basis_expectation(self, basis_state:<built-in function array>) -> float
 |      Provided a single Pauli-Z basis state, computes the expectation value of the operator
 |  
 |  __init__(self, operator:Union[List[str], Dict[str, float], <built-in function array>], coeff_vec:Union[List[complex], <built-in function array>]=None) -> None
 |  
 |  ansatz_expectation(self, ansatz_op:'AnsatzOp', basis_state:<built-in function array>, trotter_number:int=1, exact:bool=True) -> float
 |  
 |  gradient(self, ansatz_op:symred.

In [4]:
from symred.adapt_cs_vqe import ADAPT_CS_VQE
adapt = ADAPT_CS_VQE(ham, ucc, ref)

In [5]:
stab_indices, energy, ansatz = adapt.adaptive_greedy_search(7, depth=2, threshold=0.01, print_info=True)

--------------------------------------------
Using best gradient as termination criterion
--------------------------------------------

-------------------------------------------------
Searching for optimal 2-qubit contextual subspace
-------------------------------------------------

Testing stabilizer indices [2, 3, 4, 5, 6, 7, 8, 9]
>>> Ansatz term #1 gradient = 8.220633276323497e-07.
<!> The new ansatz term achieved the gradient threshold <!>

Testing stabilizer indices [1, 3, 4, 5, 6, 7, 8, 9]
>>> Ansatz term #1 gradient = 0.14214948941338434.
>>> Ansatz term #2 gradient = 0.005931132052381827.
<!> The new ansatz term achieved the gradient threshold <!>

Testing stabilizer indices [1, 2, 4, 5, 6, 7, 8, 9]
>>> Ansatz term #1 gradient = 0.0664535414242664.
>>> Ansatz term #2 gradient = 0.002508859646425776.
<!> The new ansatz term achieved the gradient threshold <!>

Testing stabilizer indices [1, 2, 3, 5, 6, 7, 8, 9]
>>> Ansatz term #1 gradient = 8.220633276323497e-07.
<!> The new

>>> Ansatz term #3 gradient = 0.09120586359294691.
>>> Ansatz term #4 gradient = 0.07058769331682413.
>>> Ansatz term #5 gradient = 0.006045013880921601.
<!> The new ansatz term achieved the gradient threshold <!>

Testing stabilizer indices [1, 3, 5, 6, 8, 9]
>>> Ansatz term #1 gradient = 0.20408051209257394.
>>> Ansatz term #2 gradient = 0.13148303241811732.
>>> Ansatz term #3 gradient = 0.06520396976037546.
>>> Ansatz term #4 gradient = 0.005080515114826767.
<!> The new ansatz term achieved the gradient threshold <!>

Testing stabilizer indices [1, 3, 4, 6, 8, 9]
>>> Ansatz term #1 gradient = 0.20408051209255973.
>>> Ansatz term #2 gradient = 0.13148303241811732.
>>> Ansatz term #3 gradient = 0.12224688383409443.
>>> Ansatz term #4 gradient = 0.004584912163309696.
<!> The new ansatz term achieved the gradient threshold <!>

Testing stabilizer indices [1, 3, 4, 5, 8, 9]
>>> Ansatz term #1 gradient = 0.20408051209257394.
>>> Ansatz term #2 gradient = 0.1314830324181031.
>>> Ansatz ter

>>> Ansatz term #7 gradient = 0.06560543837004218.
>>> Ansatz term #8 gradient = 0.03445304188475973.
>>> Ansatz term #9 gradient = 0.03247899297100787.
>>> Ansatz term #10 gradient = 0.016927678934720802.
<!> ADAPT-VQE reached the maximum number of ansatz terms <!>

Testing stabilizer indices [4, 6, 8, 9]
>>> Ansatz term #1 gradient = 0.3041938770522421.
>>> Ansatz term #2 gradient = 0.20408051209258815.
>>> Ansatz term #3 gradient = 0.1529342418602937.
>>> Ansatz term #4 gradient = 0.1529016980639426.
>>> Ansatz term #5 gradient = 0.13841287043395312.
>>> Ansatz term #6 gradient = 0.12476622896561196.
>>> Ansatz term #7 gradient = 0.10428932190002627.
>>> Ansatz term #8 gradient = 0.08502570004753807.
>>> Ansatz term #9 gradient = 0.08483727439038091.
>>> Ansatz term #10 gradient = 0.0847994700125696.
<!> ADAPT-VQE reached the maximum number of ansatz terms <!>

Testing stabilizer indices [4, 5, 8, 9]
>>> Ansatz term #1 gradient = 0.3041938770522421.
>>> Ansatz term #2 gradient = 0.2

KeyboardInterrupt: 